In [145]:
data = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_small.csv', engine='python')
data = data.sample(frac=1)

In [150]:
train = data[:3500]
test = data[3500:]

In [154]:
train.to_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', encoding='utf-8', index=False)
test.to_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', encoding='utf-8', index=False)

In [48]:
from collections import defaultdict
import time
import gc
import pandas as pd
import numpy as np

from tqdm import tqdm

s = time.perf_counter()
train = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_train.csv', engine='python')
test = pd.read_csv(r'E:\ML_study\学习笔记\推荐\movielens\ml-20m\ratings_test.csv', engine='python')

def combine(data):
    item_users = dict()
    user_items = dict()
    users = set()
    data['userId'] = 'user_' + data['userId'].astype(str)
    data['movieId'] = 'movie_' + data['movieId'].astype(str)
    for u, item, r in data[['userId', 'movieId', 'rating']].values:
        item_users.setdefault(item, []).append((u, r))
        user_items.setdefault(u, []).append((item, r))
        users.add(u)
    del data
    gc.collect()
    return item_users, user_items, users

train_item_users, train_user_items, train_users = combine(train)
test_item_users, test_user_items, test_users = combine(test)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

cost time 0.0019079916666669306 min


In [98]:
# from joblib import dump, load
# dump(item_users, r'D:\Users\hao.guo\比赛代码提炼\推荐系统\movielen\data\item_users.pkl')
# dump(users, r'D:\Users\hao.guo\比赛代码提炼\推荐系统\movielen\data\users.pkl')

In [49]:
import math
from tqdm import tqdm_notebook
def UserSimilarity(item_users, users):
    # build inverse table for item_users
#     item_users = dict()
#     users = set()
#     for u, items in data.items():
#         for i in items.keys():
#             # if i not in item_users:
#             #     item_users[i] = set()
#             item_users.setdefault(i, set()).add(u)
#         users.add(u)
    #calculate co-rated items between users
    # C = dict()
    C = defaultdict(dict)
    # C = np.zeros((len(item_users), len(item_users)))
    # user items list
    N = defaultdict(int)
    print('共现矩阵计算')
    for i, pairs in tqdm(item_users.items()):
        for u, r1 in pairs:
            N[u] += r1 ** 2
            for v, r2 in pairs:
                if u == v:
                    continue
                if v not in list(C[u]):
                    C[u][v] = 0
                C[u][v] += r1 * r2
    #calculate finial similarity matrix W
    W = defaultdict(dict)
    print('相似矩阵计算')
    for u, related_users in tqdm(C.items()):
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W
s = time.perf_counter()
W = UserSimilarity(train_item_users, train_users)
print('cost time %s min' % ((time.perf_counter() - s) / 60))

共现矩阵计算


100%|███████████████████████████████████████████████████████████████████████████| 1694/1694 [00:00<00:00, 56617.91it/s]


相似矩阵计算


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 25070.56it/s]


cost time 0.0007139449999992091 min


In [50]:
sorted(W['user_1'].items(), key=lambda p: p[1], reverse=True)[0:5]

[('user_21', 0.19303695393600245),
 ('user_3', 0.19136694720294242),
 ('user_25', 0.1847717426269238),
 ('user_24', 0.13473512787501885),
 ('user_47', 0.11828936840773875)]

In [59]:
def Recommend(user, train, W, K):
    rank = defaultdict(int)
    sim_sums = defaultdict(int)
    
    interacted_items = list(map(lambda p: p[0], train[user]))
    for v, wuv in sorted(W[user].items(), key=lambda p: p[1], reverse=True)[0:K]:
        for i, rvi in train[v]:
            if i in interacted_items:
            #we should filter items user interacted before
                continue
            rank[i] += wuv * rvi
            sim_sums[i] += wuv
    res = [(sim / 1, i) for i, sim in rank.items()]
    res.sort(reverse=True)
    ranked_items = [x[1] for x in res]
#     sorted(rank.items(), key=lambda p: p[1], reverse=True)
    return ranked_items[:50]

def Precision(train, test, W, N):
    hit = 0
    all = 0
    for user in test.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item in rank:
            if item in tu:
                hit += 1
        all += 50
    return hit / (all * 1.0)

def Recall(train, test, W, N):
    hit = 0
    all = 0
    for user in test.keys():
        tu = list(map(lambda p: p[0], test[user]))
        rank = Recommend(user, train, W, N)
        for item in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)

def Coverage(train, test, W, N):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item, r in test[user]:
            all_items.add(item)
    for user in test.keys():
        rank = Recommend(user, train, W, N)
        for item in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)

def Popularity(train, test, W, N):
    item_popularity = dict()
    for user, items in train.items():
        for item, r in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in test.keys():
        rank = Recommend(user, train, W, N)
        for item in rank:
            ret += np.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret 

def eval(train_user_items, test_user_items, W, N):
    p = Precision(train_user_items, test_user_items, W, N)
    r = Recall(train_user_items, test_user_items, W, N)
    c = Coverage(train_user_items, test_user_items, W, N)
    pop = Popularity(train_user_items, test_user_items, W, N)
    return (p, r, c, pop)
    
print(eval(train_user_items, test_user_items, W, 5))
print(eval(train_user_items, test_user_items, W, 10))
print(eval(train_user_items, test_user_items, W, 20))
print(eval(train_user_items, test_user_items, W, 30))
print(eval(train_user_items, test_user_items, W, 40))
print(eval(train_user_items, test_user_items, W, 50))

(0.1196, 0.19946631087391595, 0.534020618556701, 1.901404008414894)
(0.1316, 0.21947965310206805, 0.35670103092783506, 2.0821030298441956)
(0.1368, 0.22815210140093395, 0.25876288659793817, 2.2046577786199233)
(0.136, 0.2268178785857238, 0.22783505154639175, 2.264661699694442)
(0.1352, 0.22548365577051369, 0.2134020618556701, 2.2836800705995213)
(0.134, 0.22348232154769845, 0.2134020618556701, 2.2868977375518935)


In [58]:
Precision(train_user_items, test_user_items, W, 5)

(299, 2500)

In [52]:
len(Recommend('user_1', train_user_items, W, 5))

50

In [54]:
len(test_user_items['user_1'])

49

In [61]:
a = {1, 2, 3}
b = {2, 4, 5}
a.union(b)

{1, 2, 3, 4, 5}